In [ ]:
# Import the necessary libraries
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

# Define the path to your AID dataset
train_dir = '/content/drive/MyDrive/Dataset/train'
valid_dir = '/content/drive/MyDrive/Dataset/test'

# Load the VGG19 model
model = VGG19(weights='imagenet', include_top=True)

# Specify the layers for concatenation
layer_name1 = 'block1_pool'
layer_name2 = 'block2_pool'
layer_name3 = 'block3_pool'
layer_name4 = 'block4_pool'

# Extract the outputs of the desired layers
output1 = model.get_layer(layer_name1).output
x= MaxPooling2D(pool_size=(2, 2))(output1)
x= MaxPooling2D(pool_size=(2, 2))(x)
x= MaxPooling2D(pool_size=(2, 2))(x)
output2 = model.get_layer(layer_name2).output
x1= MaxPooling2D(pool_size=(2, 2))(output2)
x1= MaxPooling2D(pool_size=(2, 2))(x1)
x1=Conv2D(64,(1,1), activation='relu')(x1)
output3 = model.get_layer(layer_name3).output
x2= MaxPooling2D(pool_size=(2, 2))(output3)
x2=Conv2D(128,(1,1), activation='relu')(x2)
x2=Conv2D(64,(1,1), activation='relu')(x2)
output4 = model.get_layer(layer_name4).output
x3=Conv2D(256,(1,1), activation='relu')(output4)
x3=Conv2D(128,(1,1), activation='relu')(x3)
x3=Conv2D(64,(1,1), activation='relu')(x3)

# Concatenate the activations of the four layers along the channel axis (axis=-1)
concatenated_activations = tf.concat([x,x1,x2,x3], axis=-1)

x = GlobalAveragePooling2D()(concatenated_activations)

x = Dense(256, activation='relu')(x)
output = Dense(30, activation='softmax')(x)

model = Model(inputs=model.input, outputs=output)

# Data generators
train_generator = ImageDataGenerator(...).flow_from_directory(train_dir)
valid_generator = ImageDataGenerator(...).flow_from_directory(valid_dir)

# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Train model
model.fit(train_generator,validation_data=valid_generator,epochs=10)

# Save the trained model
model.save('vgg19_aid_final.h5')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 8267 images belonging to 30 classes.
Found 1800 images belonging to 30 classes.
Epoch 1/10
259/259 [==============================] - 3614s 14s/step - loss: 47.1516 - accuracy: 0.1516 - val_loss: 10.0913 - val_accuracy: 0.2272
Epoch 2/10
259/259 [==============================] - 164s 632ms/step - loss: 6.8061 - accuracy: 0.2794 - val_loss: 4.6211 - val_accuracy: 0.3100
Epoch 3/10
259/259 [==============================] - 163s 629ms/step - loss: 4.0958 - accuracy: 0.3528 - val_loss: 3.9279 - val_accuracy: 0.3261
Epoch 4/10
259/259 [==============================] - 164s 634ms/step - loss: 2.9209 - accuracy: 0.4093 - val_loss: 3.0541 - val_accuracy: 0.3894
Epoch 5/10
259/259 [==============================] - 161s 622ms/step - loss: 2.3218 - accuracy: 0.4633 - val_loss: 2.8173 - val_accuracy: 0.3928
Epoch 6/10
259/259 [==============================] - 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Import the necessary libraries
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

# Load the trained model
loaded_model = load_model('vgg19_aid_final.h5')

# Load and preprocess an image for prediction
img_path = '/content/drive/MyDrive/TESTING/farmland.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Normalize the image

# Make predictions
predictions = loaded_model.predict(img_array)

# Get the class with the highest probability
predicted_class = np.argmax(predictions)

thisdict = {
     0 : "Airport",
     1 : "BareLand",
     2 : "BaseballField",
     3 : "Beach",
     4 : "Bridge",
     5 : "Center",
     6 : "Church",
     7 : "Commercial",
     8 : "DenseResidential",
     9 : "Desert",
     10: "Farmland",
     11: "Forest",
     12: "Industrial",
     13: "Meadow",
     14: "MediumResidential",
     15: "Mountain",
     16: "Park",
     17: "Parking",
     18: "Playground",
     19: "Pond",
     20: "Port",
     21: "RailwayStation",
     22: "Resort",
     23: "River",
     24: "School",
     25: "SparseResidential",
     26: "Square",
     27: "Stadium",
     28: "Storage Tanks",
     29: "Viaduct"
}

# Print the predicted class
print(f"The image belongs to class {thisdict[predicted_class]}")

Mounted at /content/drive


OSError: ignored

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import the necessary libraries
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test data
y_pred = model.predict(valid_generator)
y_true = valid_generator.classes

# Convert predicted probabilities to class labels
y_pred_labels = tf.argmax(y_pred, axis=1)

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred_labels)
precision = precision_score(y_true, y_pred_labels, average='weighted')
recall = recall_score(y_true, y_pred_labels, average='weighted')
f1 = f1_score(y_true, y_pred_labels, average='weighted')

print("Accuracy: {:.2f} %".format(accuracy * 100))
print("Precision: {:.2f} %".format(precision * 100))
print("Recall: {:.2f} %".format(recall * 100))
print("F1 Score: {:.2f} %".format(f1 * 100))

57/57 [==============================] - 18s 323ms/step
Accuracy: 84.34 %
Precision: 83.98 %
Recall: 84.12 %
F1 Score: 84.16 %
